# What will my Math/Further Math grades be?

This notebook implements the Edexcel aggregation rules to demonstrate the A-level grades that would be obtained from a set of module scores. Simply input your scores below and let the code handle the rest!

As a guiding philosophy, the aggregation rules aim to do the following:

1. Maximise the Maths grade, then the Further Maths grade.
2. Maximise the Maths UMS, then the Further Maths UMS.

Just to be clear, the order of priority above implies that lowering your Maths UMS **(without reducing the grade)** in order to increase your Further Maths grade *is* acceptable, since grades are maximised before UMS. However, if increasing your Further Maths grade comes at the cost of reducing your Maths grade, that would *not* be acceptable, since the Math grade is maximised before the Further Maths grade.

For the sake of parsimony, this notebook will *not* do the following:

-  compute AS-level grades (it only handles the overall A-level)
-  compute Pure Maths or Additional Further Maths grades
-  handle manual requests which run against the guiding philosophy (e.g. maximising Further Maths grade before Maths grade)

*Note: Manual requests can be handled by Edexcel. No issue. But this Notebook only aims to demonstrate the standard Edexcel aggregation, not handle special requests*.

## Algorithm (based on the [Edexcel Aggregation Rules](http://furthermaths.org.uk/docs/Aggregation%20rules%20for%20HEIs%20~%20Version%203%20January%202013.pdf))

1. Allocate C1, C2, C3, C4 to Math (non-negotiable).
2. Allocate the highest two of FP1,FP2,FP3 to Further Math (non-negotiable).
3. Allocate the best of the following 6 pairs to Math:
    -  M1+M2
    -  S1+S2
    -  D1+D2
    -  M1+S1
    -  M1+D1
    -  S1+D1. 
4. Allocate the highest 4 of the remaining modules to Further Math.
5. Check if the Further Math grade can be improved on by allocating the best pair from step (3) to Further Math and the second best pair to Math, *subject to not lowering the Math grade*.
6. If it can, then the re-allocation is performed. If it cannot, then the allocation in (4) is already optimal.

In [10]:
import pandas as pd
import numpy as np
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import HBox, VBox
from IPython.display import Javascript, display, Math, Latex

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Initialise")
button.on_click(run_all)
display(button)

<IPython.core.display.Javascript object>

A Jupyter Widget

In [11]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to had
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

pandas==0.22.0
numpy==1.14.0
ipywidgets==7.0.0


In [12]:
# get score from master list
def getscore (str, list):
    return list.index(str)

# get highest two further pure modules
def fpure(fpure):
    
    if fpure.index(min(fpure)) == 2: return "FP1", "FP2"
    elif fpure.index(min(fpure)) == 1: return "FP1", "FP3"
    else : return "FP2", "FP3"

# compute maths grade
def grade_maths(maths,scores):
    
    maths_scores = [0,0,0,0,0,0]
    
    for i in range(0,6):
        maths_scores[i] = scores[1][getscore(maths[i], scores[0])]
    
    total = sum(maths_scores)
    star = maths_scores[2] + maths_scores[3]
    
    result = 0
    if total < 240: result = 7
    elif total < 300: result = 6
    elif total < 360: result = 5
    elif total < 420: result = 4
    elif total < 480: result = 3
    elif total >= 480 and star < 180: result = 2
    elif total >= 480 and star >= 180: result = 1
        
    return result

# compute further maths grade
def grade_fmaths(fmaths,scores):
    
    fmaths_scores = [0,0,0,0,0,0]
    for i in range(0,6):
        fmaths_scores[i] = scores[1][getscore(fmaths[i], scores[0])]
    
    total = sum(fmaths_scores)
    
    star_scores = fmaths_scores
    for i in range (0,6):
        if "1" in fmaths[i]: star_scores[i]=0
    star = sorted(star_scores)[-1] + sorted(star_scores)[-2] + sorted(star_scores)[-3]
    
    result = 0
    if total < 240: result = 7
    elif total < 300: result = 6
    elif total < 360: result = 5
    elif total < 420: result = 4
    elif total < 480: result = 3
    elif total >= 480 and star < 270: result = 2
    elif total >= 480 and star >= 270: result = 1
        
    return result

# compute grade for output
def grade_output(int):
    
    result = ''
    if int == 7: result = 'U'
    elif int == 6: result = 'E'
    elif int == 5: result = 'D'
    elif int == 4: result = 'C'
    elif int == 3: result = 'B'
    elif int == 2: result = 'A'
    elif int == 1: result = 'A*'
        
    return result

In [17]:
def f(C1=0, C2=0, C3=0, C4=0, M1=0, M2=0, M3=0, S1=0, S2=0, S3=0, D1=0, D2=0, FP1=0, FP2=0, FP3=0, M4=0, M5=0, S4=0):
    
#first, get a master copy
    scores = [["C1","C2","C3","C4","FP1","FP2","FP3","M1","M2","M3","M4","M5","S1","S2","S3","S4","D1","D2",""], 
              [C1, C2, C3, C4, FP1, FP2, FP3, M1, M2, M3, M4, M5, S1, S2, S3, S4, D1, D2,0]]
    
# insert non-negotiable scores
    maths = ["C1","C2","C3","C4", "", ""]
    maths_scores = [0,0,0,0,0,0]
    
    fmaths1, fmaths2 = fpure([FP1,FP2,FP3])
    fmaths = [fmaths1,fmaths2,"", "","",""]
    fmaths_scores = [0,0,0,0,0,0]
    
# make best assignment to math
    maths_posib = [["M1", "M2"],["D1", "D2"],["S1", "S2"], ["M1", "S1"], ["M1", "D1"], ["S1", "D1"]]
    temp = [0,1,2,3,4,5]
    maths_posib_scores = [0,0,0,0,0,0]
    for i in range (0,6):
        maths_posib_scores[i] = scores[1][getscore(maths_posib[i][0],scores[0])] + scores[1][getscore(maths_posib[i][1],scores[0])]
    maths_posib_sorted = [x for _,x in sorted(zip(maths_posib_scores,temp))]
    
    maths[4] = maths_posib[maths_posib_sorted[5]][0]
    maths[5] = maths_posib[maths_posib_sorted[5]][1]
    
#format and display output
    for i in range(0,6):
        maths_scores[i] = scores[1][getscore(maths[i], scores[0])]
        
    for i in range(0,6):
        fmaths_scores[i] = scores[1][getscore(fmaths[i], scores[0])]
    
    math_output = pd.DataFrame({'Module': maths, 'Score':maths_scores})
    fmath_output = pd.DataFrame({'Module': fmaths, 'Score':fmaths_scores})
    
    print("Maths grade: " + grade_output(grade_maths(maths, scores)))
    print(math_output.to_string(index=False))
    print("")
    print("Further maths grade: " + grade_output(grade_fmaths(fmaths, scores)))
    print(fmath_output.to_string(index=False))

In [18]:
C1 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='C1:')
C2 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='C2:')
C3 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='C3:')
C4 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='C4:')
C = HBox([C1,C2,C3,C4])

FP1 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='FP1:')
FP2 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='FP2:')
FP3 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='FP3:')
FP = HBox([FP1,FP2,FP3],layout={'width':'75%'})

M1 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='M1:')
M2 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='M2:')
M3 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='M3:')
M4 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='M4:')
M5 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='M5:')
M = HBox([M1,M2,M3,M4,M5], layout={'width':'125%'})

S1 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='S1:')
S2 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='S2:')
S3 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='S3:')
S4 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='S4:')
S = HBox([S1,S2,S3,S4])

D1 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='D1:')
D2 = widgets.IntSlider(value=0, min = 0, max = 100, step = 1, description='D2:')
D = HBox([D1,D2], layout={'width':'50%'})

your_marks = widgets.VBox([C,FP,M,S,D])
your_grades = widgets.interactive_output(f, {'C1':C1,'C2':C2,'C3':C3,'C4':C4,'FP1':FP1,'FP2':FP2,'FP3':FP3,'M1':M1,'M2':M2,'M3':M3,'M4':M4,'M5':M5,'S1':S1,'S2':S2,'S3':S3,'S4':S4,'D1':D1,'D2':D2})

display(your_marks)
print('Resuts below:')
display(your_grades)

A Jupyter Widget

Resuts below:


A Jupyter Widget